In [ ]:
# You may need to run this the first time !pip install lorem confluent_kafka sseclient

In [ ]:
!pip install lorem confluent_kafka sseclient

In [5]:
from confluent_kafka import SerializingProducer, DeserializingConsumer
from confluent_kafka.serialization import StringSerializer, StringDeserializer
from confluent_kafka.admin import AdminClient, NewTopic
from uuid import uuid4
import sys, lorem, random

brokers = "kafka1:9092"

In [6]:
pconf = {
    'bootstrap.servers': brokers,
    'partitioner': 'murmur2_random',
    'key.serializer': StringSerializer('utf_8'),
    'value.serializer': StringSerializer('utf_8')
}

In [7]:
p = SerializingProducer(pconf)

In [ ]:
import time
for n in range(1,1000):
    try:
        # Produce line (without newline)
        line = lorem.sentence()
        print(line)
        p.produce("words", key=str(uuid4()), value=line)
        p.poll(0)
        p.flush()
        print(line)
    except BufferError:
        sys.stderr.write('%% Local producer queue is full (%d messages awaiting delivery): try again\n' % len(p))

In [ ]:
import time
for n in range(1,1000):
    try:
        # Produce line (without newline)
        line = lorem.sentence()
        p.produce("words", key=str(uuid4()), value=line)
        p.poll(0)
        p.flush()
        time.sleep(0.5)
        print(line)
    except BufferError:
        sys.stderr.write('%% Local producer queue is full (%d messages awaiting delivery): try again\n' % len(p))

## Wikipedia edit stream

In [8]:
user_types = {True: 'bot', False: 'human'}
def init_namespaces():
    # create a dictionary for the various known namespaces
    # more info https://en.wikipedia.org/wiki/Wikipedia:Namespace#Programming
    namespace_dict = {-2: 'Media', 
                      -1: 'Special', 
                      0: 'main namespace', 
                      1: 'Talk', 
                      2: 'User', 3: 'User Talk',
                      4: 'Wikipedia', 5: 'Wikipedia Talk', 
                      6: 'File', 7: 'File Talk',
                      8: 'MediaWiki', 9: 'MediaWiki Talk', 
                      10: 'Template', 11: 'Template Talk', 
                      12: 'Help', 13: 'Help Talk', 
                      14: 'Category', 15: 'Category Talk', 
                      100: 'Portal', 101: 'Portal Talk',
                      108: 'Book', 109: 'Book Talk', 
                      118: 'Draft', 119: 'Draft Talk', 
                      446: 'Education Program', 447: 'Education Program Talk', 
                      710: 'TimedText', 711: 'TimedText Talk', 
                      828: 'Module', 829: 'Module Talk', 
                      2300: 'Gadget', 2301: 'Gadget Talk', 
                      2302: 'Gadget definition', 2303: 'Gadget definition Talk'}

    return namespace_dict


def construct_event(event_data, user_types):
    # use dictionary to change assign namespace value and catch any unknown namespaces (like ns 104)
    try:
        event_data['namespace'] = namespace_dict[event_data['namespace']]
    except KeyError:
        event_data['namespace'] = 'unknown'

    # assign user type value to either bot or human
    user_type = user_types[event_data['bot']]

    # define the structure of the json event that will be published to kafka topic
    event = {"id": event_data['id'],
             "domain": event_data['meta']['domain'],
             "uri": event_data['meta']['uri'],
             "namespace": event_data['namespace'],
             "title": event_data['title'],
             #"comment": event_data['comment'],
             "timestamp": event_data['meta']['dt'],#event_data['timestamp'],
             "user_name": event_data['user'],
             "user_type": user_type
             #"minor": event_datab['minor'],
             }
    
    if event_data["type"] == "edit":
        event["old_length"] = event_data['length']['old']
        event["new_length"] = event_data['length']['new']
    
    return event

In [12]:
!pip install requests


[notice] A new release of pip is available: 23.2.1 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [13]:
import json
import requests
from sseclient import SSEClient as EventSource

url = 'https://stream.wikimedia.org/v2/stream/recentchange'
namespace_dict = init_namespaces()
res = requests.get(url)

print(res)
for event in EventSource(url):
    if event.event == 'message':
        try:
            event_data = json.loads(event.data)
        except ValueError:
            pass
        else:
            if event_data['type'] == 'edit':
                    # construct valid json event
                    event_to_send = construct_event(event_data, user_types)
                    print(event_to_send)
                    p.produce('edit', value=json.dumps(event_to_send))
                    p.poll(0)
                    p.flush()

<Response [403]>


HTTPError: 403 Client Error: Forbidden for url: https://stream.wikimedia.org/v2/stream/recentchange

In [16]:
import json
import requests
from sseclient import SSEClient  # pip install sseclient-py

URL = "https://stream.wikimedia.org/v2/stream/recentchange"

# Wikimedia requires a descriptive User-Agent with a contact
session = requests.Session()
session.headers.update({
    "User-Agent": "MyWikiStreamBot/0.1 (andrea.mauri@univ-lyon1.fr)",  # <-- put your contact
    "Accept": "text/event-stream",
})

# Connect directly via SSEClient using the session
client = SSEClient(URL, session=session)

for event in client:
    if event.event != "message" or not event.data:
        continue
    try:
        data = json.loads(event.data)
    except ValueError:
        continue
    if data.get("type") == "edit":
        # build your event; placeholders shown
        event_to_send = construct_event(data, user_types)
        print(event_to_send)
        p.produce('edit', value=json.dumps(event_to_send))
        p.poll(0)
        p.flush()
        

{'id': 372404545, 'domain': 'de.wikipedia.org', 'uri': 'https://de.wikipedia.org/wiki/Landtags-_und_Gemeinderatswahlkreis_Landstra%C3%9Fe', 'namespace': 'main namespace', 'title': 'Landtags- und Gemeinderatswahlkreis Landstraße', 'timestamp': '2025-10-24T13:59:47.999Z', 'user_name': 'Stefan Fröhlich', 'user_type': 'human', 'old_length': 7879, 'new_length': 8347}
{'id': 3060875625, 'domain': 'commons.wikimedia.org', 'uri': 'https://commons.wikimedia.org/wiki/File:11-01-15-buchmesse-ffm-by-RalfR-11.jpg', 'namespace': 'File', 'title': 'File:11-01-15-buchmesse-ffm-by-RalfR-11.jpg', 'timestamp': '2025-10-24T13:59:48.041Z', 'user_name': 'MB-one', 'user_type': 'human', 'old_length': 8153, 'new_length': 8110}
{'id': 2495602792, 'domain': 'www.wikidata.org', 'uri': 'https://www.wikidata.org/wiki/Q10791288', 'namespace': 'main namespace', 'title': 'Q10791288', 'timestamp': '2025-10-24T13:59:48.228Z', 'user_name': 'Peter James', 'user_type': 'human', 'old_length': 2871, 'new_length': 2781}
{'id':

KeyError: 'user'

In [ ]:
import json
from sseclient import SSEClient as EventSource

url = 'https://stream.wikimedia.org/v2/stream/recentchange'
namespace_dict = init_namespaces()
for event in EventSource(url):
    if event.event == 'message':
        try:
            event_data = json.loads(event.data)
        except ValueError:
            pass
        else:
            if "type" in event_data and event_data['type'] != 'log':
                # construct valid json event
                event_to_send = construct_event(event_data, user_types)
                print(event_to_send)
                p.produce(event_data['type'], value=json.dumps(event_to_send))
                p.poll(0)
                p.flush()